In [47]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
df = pd.read_csv("cust_satisfaction.csv")
df.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied,13,460,5,4,5,25,18.0
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied,25,235,1,3,1,1,6.0
2,Female,Loyal Customer,Business travel,Business,satisfied,26,1142,5,4,5,0,0.0
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied,25,562,2,3,2,11,9.0
4,Male,Loyal Customer,Business travel,Business,satisfied,61,214,3,4,3,0,0.0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      103904 non-null  object 
 1   Customer Type               103904 non-null  object 
 2   Type of Travel              103904 non-null  object 
 3   Class                       103904 non-null  object 
 4   satisfaction                103904 non-null  object 
 5   Age                         103904 non-null  int64  
 6   Flight Distance             103904 non-null  int64  
 7   Inflight entertainment      103904 non-null  int64  
 8   Baggage handling            103904 non-null  int64  
 9   Cleanliness                 103904 non-null  int64  
 10  Departure Delay in Minutes  103904 non-null  int64  
 11  Arrival Delay in Minutes    103594 non-null  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 9.5+ MB


In [50]:
df.isnull().sum()
df.dropna(inplace=True)

In [51]:

df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [52]:
df["Customer Type"].value_counts()

Customer Type
Loyal Customer       84517
disloyal Customer    18905
Name: count, dtype: int64

In [53]:
loyal_customer = df[df["Customer Type"] == "Loyal Customer"]
disloyal_customer = df[df["Customer Type"] == "disloyal customer"]

In [54]:
loyal_customer = loyal_customer.sample(20000)
loyal_customer.shape

(20000, 12)

In [55]:
balance_df = pd.concat([loyal_customer,disloyal_customer],axis=0)
balance_df.shape

(20000, 12)

In [56]:
cat_col = balance_df.select_dtypes(include=["object"])
cat_col.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction
20085,Male,Loyal Customer,Business travel,Business,satisfied
56695,Female,Loyal Customer,Business travel,Eco,satisfied
98906,Male,Loyal Customer,Business travel,Business,neutral or dissatisfied
23410,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
10164,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied


In [57]:
num_col = balance_df.select_dtypes(exclude=["object"])
num_col.head()

,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
20085,51,1250,2,2,4,0,0.0
56695,25,2355,4,4,4,0,4.0
98906,65,1379,2,2,2,4,1.0
23410,65,920,4,3,3,0,0.0
10164,45,192,1,1,3,0,0.0


In [58]:
cat_col["Customer Type"].value_counts()

Customer Type
Loyal Customer    20000
Name: count, dtype: int64

In [62]:
pd.get_dummies(cat_col,drop_first=True).astype(int).head()


,Gender_Male,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,satisfaction_satisfied
20085,1,0,0,0,1
56695,0,0,1,0,1
98906,1,0,0,0,0
23410,0,1,1,0,0
10164,0,1,1,0,0


In [65]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ohe = OneHotEncoder(drop="if_binary")
cat_col_encoded = ohe.fit_transform(cat_col).toarray()
cat_col_encoded

array([[1., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 0., 1.]], shape=(20000, 7))

In [66]:
column_name = list(ohe.get_feature_names_out())
column_name

['Gender_Male',
 'Customer Type_Loyal Customer',
 'Type of Travel_Personal Travel',
 'Class_Business',
 'Class_Eco',
 'Class_Eco Plus',
 'satisfaction_satisfied']

In [67]:
one_hot = pd.DataFrame(cat_col_encoded,columns=column_name)
one_hot.head()

,Gender_Male,Customer Type_Loyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0
4,0.0,1.0,1.0,0.0,1.0,0.0,0.0


In [68]:
one_hot = one_hot.reset_index(drop=True)
num_col = num_col.reset_index(drop=True)
final_df = pd.concat([one_hot,num_col],axis=1)
final_df.head()

,Gender_Male,Customer Type_Loyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,51,1250,2,2,4,0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,25,2355,4,4,4,0,4.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,65,1379,2,2,2,4,1.0
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,65,920,4,3,3,0,0.0
4,0.0,1.0,1.0,0.0,1.0,0.0,0.0,45,192,1,1,3,0,0.0


In [70]:
##x and y ----> train test split --> algo train

x = final_df.drop("satisfaction_satisfied",axis=1)
y = final_df["satisfaction_satisfied"]


